# Libraries

In [ ]:
from bs4 import BeautifulSoup
from requests import Session

# Login

Intentaremos obtener información de "Mi Portal UC - Retenciones del Estudiante". Sin embargo, acceder a esta página nos va a redirigir al SSO.

In [ ]:
session = Session()
url_we_want_to_scrape = 'https://portal.uc.cl/c/portal/render_portlet?p_l_id=1534992&p_p_id=RetencionesAlumno_WAR_LPT034_RetencionesAlumnoportlet&p_p_lifecycle=0&p_t_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-2&p_p_col_pos=2&p_p_col_count=6&p_p_isolated=1&currentURL=%2Fweb%2Fcomunidad-uc%2Finformacion-academica'
initial_response = session.get(url=url_we_want_to_scrape)
login_url = initial_response.url

print(f'Queremos scrapear {url_we_want_to_scrape},')
print(f'pero fuimos redirigidos a {login_url}.')

Rellenamos el formulario de inicio de sesión como lo hacemos usualmente, enviando nuestro `username` y `password`.

In [ ]:
username = input('Username: ')
password = input('Password: ')

Pero también tenemos que incluir los campos *hidden*, así que le hacemos `GET` al formulario y extraemos esos campos. Deberían aparecer entre ellos `execution`, cuyo valor es un ID, y `_eventId`, cuyo valor debería ser "submit".

In [ ]:
response = session.get(url=login_url)
hidden_input_fields = BeautifulSoup(response.text, 'html.parser').find_all('input', type='hidden')
hidden_input_fields

El form resultante es el siguiente.

In [ ]:
form_data = {input_field.get('name'): input_field.get('value') for input_field in hidden_input_fields}
form_data['username'] = username
form_data['password'] = password
form_data

Ahora, cuando hagamos `POST` de este form, se debería inicial sesión y el SSO debería redirigirnos a la página original, que era "Mi Portal UC - Retenciones del Estudiante".

In [ ]:
response = session.post(url=login_url, data=form_data)
print(f'URL final: {response.url}')

# Scraping

Finalmente, le hacemos scraping a la página que recibimos.

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
rows = soup.find_all('tr')
for row in rows:
    columns = row.find_all('td')
    for column in columns:
        if column.string:
            print(column.string)
    print()